In [ ]:
import pandas as pd
import re as re

In [ ]:
ingredients = pd.read_csv('ingredient_info_processed.csv')
products = pd.read_csv('skincare_products_processed.csv')

In [ ]:
def ingredient_in_INCI(inci, ingredient_name):
    cleaned_inci_text = re.sub(r'[^\w\s]', ' ', inci).lower()
    words_in_inci = set(cleaned_inci_text.split())
    cleaned_ingredient_text = re.sub(r'[^\w\s]', ' ', ingredient_name).lower()
    words_in_name = set(cleaned_ingredient_text.split())

    return (words_in_name <= words_in_inci)


def get_shortest_INCI(df, ingredient_name):
    '''
    Takes the ingredients dataframe (df) and an ingredient name
    as arguments, then returns the matching row from the dataframe if
    a match is found. Otherwise, returns an empty row.
    '''
    filtered_df = \
        df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name))]

    if filtered_df.size == 0:
        ingredient_name_clean = re.sub(r'\([^)]*\)', '', ingredient_name)

        filtered_df = \
            df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name_clean))]

        if filtered_df.size == 0:
            return filtered_df

    shortest_index = filtered_df['INCI name'].str.len().idxmin()
    return filtered_df.loc[[shortest_index]]


In [ ]:
def combine_ingredients_and_info(df):
  '''
  Takes the ingredients dataframe as argument then adds INCI name, COSING
  Ref No, Description, and Function of matching ingredients to the original
  ingredients dataframe.
  '''
  for index, row in df.iterrows():
    ingredient_name = str(row['INCI name'])
    INCI_details = get_shortest_INCI(df, ingredient_name)
    df['INCI name'] = INCI_details['INCI name']
    df['COSING Ref No'] = INCI_details['COSING Ref No']
    df['Chem/IUPAC Name / Description'] = INCI_details['Chem/IUPAC Name / Description']
    df['Function'] = INCI_details['Function']
  return df

In [ ]:
combine_ingredients_and_info(ingredients)

TypeError: expected string or bytes-like object